# Answer to Joe's question --- Part III: Putting it all together

In [1]:
#include <stdlib.h>
#include <limits.h>
#include <iostream>

struct node {
    struct node* left;
    struct node* right;
    struct node* parent;
    long         tip; // Indicates node is a tip, also assigns a unique tip number
    long         poIndex; // Index of node assigned in postorder
    long         memIndex; // Denotes the position of the node within the tree's node array
};

typedef struct node Node; // 

struct tree {
    long   numTips;  // Determined by input data at run-time.
    long   numNodes; // Always at least 2*numTips - 1 for bifurcating trees.
    Node*  nodes;    // We don't know how many nodes our tree will be at compile time, 
                     //so we are reserving a pointer that will point to a block of nodes once we know what we need
    Node*  root;     // A helpful pointer that gives us a starting point for traversals on our tree
};

typedef struct tree Tree; // As with Node, we can now use Tree as a valid CXX data type

In [2]:
char char2DNAbase(const char c)
{
    if (c == 'A' || c == 'a') {
        return (char)1;
    }
    else if (c == 'C' || c == 'c') {
        return (char)1 << 1;
    }
    else if (c == 'G' || c == 'g') {
        return (char)1 << 2;
    }
    else if (c == 'T' || c == 't') {
        return (char)1 << 3;
    }
    else if (c == '?' || c == '-') {
        return CHAR_MAX; // In other words: 11111111
    }
    
    return -1; // A nonsense value for an error check
}

In [3]:
long unord_parsimony_downpass(long left, long right, long parent, char** data, long nchars)
{
    int i = 0;
    int res = 0;
    
    for (i = 0; i < nchars; ++i) {
        if (data[left][i] & data[right][i]) { // If any states in common between descendant sets...
            data[parent][i] = data[left][i] & data[right][i]; // parent set is the intersection of sets (bitwise AND)
        }
        else {
            data[parent][i] = data[left][i] | data[right][i]; // parent is the union of descendant sets (bitwise OR)
            ++res; // Add a step to the tree
        }
    }
    
    return res; // Return the number of evolutionary steps implied at this node
}

In [4]:
Tree* new_tree(int numTips)
{
    Tree* newTree = NULL;
    
    newTree = (Tree*)calloc(1, sizeof(Tree)); // Reserve a block of memory big enough for a tree struct, 
                                                // return a pointer to it so we know where that block lives
    
    // Always check returns from malloc or calloc
    if (!newTree) { //Or: "if (newTree = NULL)"
        exit(EXIT_FAILURE); // Crash the program deliberately to prevent anything stupid from happening.
                            // There are more elegant things you can do, but that's a more advanced topic.
    }
    
    // Now that that succeeded, let's set up the basic parameters of the tree:
    newTree->numTips = numTips;
    newTree->numNodes = 2 * numTips - 1;
    
    // Now we have enough information to allocate the block of nodes
    newTree->nodes = (Node*)calloc(newTree->numNodes, sizeof(Node));
    
    if (!newTree->nodes) {
        free(newTree); // Return the Tree memory to the system
        exit(EXIT_FAILURE);
    }
    
    // Let's set the memory indices in our nodes while we are at it, and number the tips
    int i = 0;
    for (i = 0; i < newTree->numNodes; ++i) {
        newTree->nodes[i].memIndex = i;
        if (i < numTips) {
            newTree->nodes[i].tip = i + 1; // Remember: we want non-zero values for tips
        }
    }
    
    return newTree;
}

In [5]:
Tree* make_tree_from_table(long edgetable[], int numtaxa)
{
    // This function will build the tree from a given table.
    
    Tree* t = new_tree(numtaxa);
     
    int i = 0;
    for (i = 0; i < (t->numNodes - 1); ++i) { // Notice we will finish before the end of the array
        int index = edgetable[i];
        t->nodes[i].parent = &t->nodes[index];
    }
    
    // The last node in the array is the root, so we don't need to point it at anything
    t->nodes[i].parent = NULL;
    t->root = &t->nodes[i]; // Point to the root while we're at it.
    
    // Now we need some clever way to hook up the descendant pointers 'left' and 'right':
    // We need only operate on the internal nodes for this:
    
    for (i = t->numTips; i < t->numNodes; ++i) {
        
        // Set up the left pointer by looking for the first index with i as its ancestor:
        int j = 0;
        while (edgetable[j] != i) {
            ++j;
        }
        t->nodes[i].left = &t->nodes[j];
        
        ++j; // Increment j one more time to move it past the current index   
       
        // Set up the right pointer
        while (edgetable[j] != i) {
            ++j;
        }
    
        t->nodes[i].right = &t->nodes[j];
    }
    
    return t;
}

In [6]:
void postorder_internal(Node* n, Node** polist, int* index)
{
    if (n->tip) {
        return;
    }
    
    postorder_internal(n->left, polist, index);
    postorder_internal(n->right, polist, index);
    
    polist[*index] = n;
    (*index)++;
    
    // Some code to verify the output, but wouldn't normally be included:
    std::cout << "Node: " << n->memIndex << " index: " << *index << std::endl;
}

In [7]:
int numtaxa = 5;
long edgetable[] = {5, 6, 7, 6, 5, 8, 7, 8, -1}; // Same tree as Part I

In [8]:
int num_sites = 5; // The number of DNA sites in the data matrix;
char rawalignment[] = "TTTAA" 
                      "TTT-C"
                      "T?TTC"
                      "TT--A"
                      "TGTTG";

In [9]:
Tree* t = NULL;
t = make_tree_from_table(edgetable, numtaxa);

In [10]:
char **nodeData = NULL; // This is our big matrix for all nodal sets in our tree

// Notice below we are using 2 * num_taxa -1 so that we can also store results for calculations on the internal nodes
nodeData = (char**)calloc(2*numtaxa - 1, sizeof(char*)); // Allocate the pointers to the rows of data

// We can now think of nodeData as something like an array of arrays (really, it's a block of memory for pointers
// to blocks of pointers to char). That means we can loop over each element in nodeData and point it at a new block
// of memory corresponding to the width of our matrix:
int i = 0;

for (i = 0; i < 2 * numtaxa - 1; ++i) {
    nodeData[i] = (char*)calloc(num_sites, sizeof(char));
}

In [11]:
for (i = 0; i < numtaxa; ++i) {
    int j = 0;
    for (j = 0; j < num_sites; ++j) {
        nodeData[i][j] = char2DNAbase(rawalignment[i * numtaxa + j]);
    }
}

In [12]:
// Declare a pointer to storage for our postorder list of nodes
Node** postorder = NULL;

In [13]:
// Determine number of nodes:
int num_nodes = 0;
num_nodes = 2 * numtaxa - 1;

In [14]:
// We only need internal nodes for our work so numtaxa-1 should be fine
postorder = (Node**)calloc(numtaxa - 1, sizeof(Node*)); // This declares a block of node pointers.
if (postorder == NULL) {
    std::cout << "Uh oh! Allocation failure!" << std::endl;
}

In [15]:
// Let's traverse the tree!
i = 0;
postorder_internal(t->root, postorder, &i);

Node: 5 index: 1
Node: 6 index: 2
Node: 7 index: 3
Node: 8 index: 4


In [16]:
for (i = 0; i < numtaxa-1; ++i) {
    std::cout << postorder[i]->memIndex << " ";
}

5 6 7 8 

In [17]:
Node* n = NULL;

// Assume here that nodes_in_postorder has been allocated and set by a traversal

int len = 0; // Length of the tree

for (i = 0; i < numtaxa - 1; ++i) {
    n = postorder[i];
    len += unord_parsimony_downpass(n->left->memIndex, n->right->memIndex, n->memIndex, nodeData, num_sites);
}

std::cout << "The length of the tree is: " << len << std::endl;

The length of the tree is: 5
